In [ ]:

'''
Key:
    NVBB7 CF-
    NVBD2 Cre;Fl/Fl
    NVAH2 Cre;Fl/Fl
    NVAH8 Cre;Fl/Fl
    NVAK6 Cre;Fl/Fl
    NVAK1 CF-
    VBN4 CF-
    NVAK7 CF- - excluded due to poor cell segmentation - clear outlier in results secondary to this
    NVAH7 CF-
    NVAK9 Cre;Fl/Fl
    NVAK4 NoCre
    NVAK5 NoCre
    NVAH1 NoCre
    NVAH5 NoCre
    NVAH6 NoCre
'''


import os
import numpy as np
import time, os, sys
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils
import glob

basedir = os.getcwd()
print(basedir)
# REPLACE FILES WITH YOUR IMAGE PATHS
files = sorted(glob.glob(basedir + '/Images_for_analysis/' + '*.tif'))
print(files)
# ['Surface of NVAH8_20Som.tif','Surface of NVAH4-1.tif',
#         'Surface of NVAH2-1 (1).tif','Sueface of NVBD2 (1).tif',
#         'NVAK6_DriftCorrectedSurface (1).tif',
#         'MAX_Reslice of Result of Reslice of NVAK1_19Som (1).tif',
#         'Analysed Surface of NVBB7-2 (1).tif']
  
imgs = [skimage.io.imread(f) for f in files]
zo1_imgs = []
gfp_imgs = []
nimg = len(imgs)
cre = ['Cre;Fl/-', 'NoCre','Cre;Fl/-','Cre;Fl/-','NoCre','Cre;Fl/Fl','Cre;Fl/Fl','NoCre','Cre;Fl/Fl','NoCre','NoCre','Cre;Fl/Fl','Cre;Fl/Fl','Cre;Fl/-']
plt.figure(figsize=(20,20))
for k,img in enumerate(imgs):
    plt.subplot(1,len(imgs),k+1)
    if img.ndim == 2:
        plt.imshow(img[:,:])
        zo1_imgs.append(img)
        gfp_imgs.append(np.zeros(img.shape))
        
    elif img.ndim == 3:
        plt.imshow(img[1,:,:])
        zo1_imgs.append(img[1,:,:])
        gfp_imgs.append(img[0,:,:])


In [ ]:
# RUN CELLPOSE


from cellpose import models, io

# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=False, model_type='cyto')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
channels = [2, 0]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
#channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images
masks, flows, styles, diams = model.eval(zo1_imgs, channels=channels)

# save results so you can load in gui
io.masks_flows_to_seg(imgs, masks, flows, diams, files, channels)

# save results as png
io.save_to_png(imgs, masks, flows, files)

In [ ]:
# DISPLAY RESULTS
from cellpose import plot

nimg = len(imgs)
for idx in range(nimg):
    maski = masks[idx]
    flowi = flows[idx][0]
    fig = plt.figure(figsize=(20,20))
    plot.show_segmentation(fig, zo1_imgs[idx], maski, flowi, channels=channels)
    plt.tight_layout()
    plt.show()

In [ ]:
# Convert to dictprintprint

from collections import defaultdict

imgd = defaultdict(dict)
for i, j, x, y, z in zip(files, cre, gfp_imgs, zo1_imgs, masks):
    imgd[i] = j, x, y, z

values = imgd.values()
keys = imgd.keys()
values_list = list(values)
keys_list = list(keys)

In [ ]:
from skimage.measure import label, regionprops, regionprops_table
import math
from skimage.segmentation import clear_border
from skimage.exposure import rescale_intensity
from scipy import ndimage

all_ind = []
gfp_ind = []
nogfp_ind = []

neighbour = []
gfp_neighbour = []

for k in imgd:
#k = list(imgd.keys())[0]
    print(k)
    print(imgd[k][0])
    maski_orig = imgd[k][3]
    maski = clear_border(imgd[k][3], buffer_size=30,bgval=0, in_place=False)
    gfp_img_unscaled = imgd[k][1]
    zo1_img = imgd[k][2]

    #rescale gfp channel to 0 - 1 float
    gfp_img = rescale_intensity(gfp_img_unscaled.astype(float),in_range='image')

    #find unique cell IDs in mask
    uniqueValues = np.unique(maski)
    #print(len(uniqueValues))
    #initialise gfp pos/neg mask
    mask_posneg = np.zeros(maski.shape)

    # initialise outlines array
    outlines_arr = np.zeros(maski.shape)

    outlines = plot.outlines_list(maski)
    # unique outlines values

    i = 1
    #print(outlines)
    for o in outlines:
        for p in o:
            #print(p)
            outlines_arr[p[1],p[0]] = uniqueValues[i]
        i += 1
    print(np.amax(outlines_arr))
    print(np.amax(maski))
    # intialise cell outlines array
    cell_outline = np.zeros(maski.shape)

    # initialise adjacency matrix
    adj = np.zeros((len(uniqueValues), len(uniqueValues)))

    # initialise EGFP list
    gfp = np.zeros(len(np.unique(maski))).astype(int)

    #rather than just using the complement of gfp to define gfp cells, we can define cells that we are confident that the cells are tdTom i.e. very little or no signal in gfp channel at all.
    # loop through uniquevalues of mask
    for j in range(len(uniqueValues)):
        # isolate individual cell mask
        cell_mask = maski == uniqueValues[j]
        # mask gfp channel by cell mask
        cell = np.where(cell_mask == True, gfp_img, np.nan)
        # find cell mask size
        cell_size = np.count_nonzero(~np.isnan(cell))

        # find number of pixels greater than an empirical threshold
        gfp_size = np.count_nonzero(cell > 0.08)
        nogfp_size = np.count_nonzero(cell < 0.06)

        # class cell as EGFP (3), confidently negative (1), or likely negative (2) depending on whether pixels are EGFP or negative
        if gfp_size >= cell_size*0.75:
            mask_posneg[maski == uniqueValues[j]] = 3
            gfp[j] = 3 # value denotes EGFP cell
            #print('cell is EGFP')

        elif nogfp_size >= cell_size*0.75: 
            mask_posneg[maski == uniqueValues[j]] = 1 # confident negative cell
            gfp[j] = 1 # value denotes a non-positive cell
            #print('cell is confidently tdTom')

        else:
            mask_posneg[maski == uniqueValues[j]] = 2 # likely negative cell
            gfp[j] = 2 # value denotes a likely non-positive cell
            #print('cell is likely tdTom')

        # define cell outline
        cell_label = np.where( maski == uniqueValues[j], 1, 0)
        # dilate cell outline
        #cell_dil = cell_label
        cell_dil = ndimage.binary_dilation(cell_label,iterations=3)

        # find neighbouring outlines 
        #print(np.amax(maski))
        masked_outlines = np.where(cell_dil == True,maski,0)
        neighbours = np.unique(masked_outlines)

        for k in neighbours:
            #print(k)
            if k != 0:
                m = np.argwhere(uniqueValues == k)
                #print(m[0][0])
                if m[0][0] != None:
                    #print(j, m[0][0])
                    adj[j, m[0][0]] = 1

    adj_sym = np.maximum( adj, adj.transpose() ).astype(int)    

    # indices of EGFP cells

    gfp_ind_indiv = np.where(gfp == 3)
    
    #print(gfp_ind_indiv)    
    
    num_neighbours = np.zeros(len(np.unique(maski))).astype(int)
    num_gfp_neighbours = np.zeros(len(np.unique(maski))).astype(int)
    
    for j in range(len(uniqueValues)):
        #if gfp[j] == 1: #this means the cell in question is confidently tdTom
        
        num_gfp_neighbours[j] = np.count_nonzero(adj_sym[gfp_ind_indiv,j])
        
        #print(indiv_num_gfp_neighbours)
        num_neighbours[j] = np.count_nonzero(adj_sym[:,j])
    
        
        #print(num_neighbours[j])
        #print(gfp_num_neighbours[j])
        #else:
            #print('cell number ' + str(j) + ' is etdTom and is not a neighbour of EGFP cell ')
        #    gfp[j] = 0 # 0 now denotes tdTom cell with no EGFP neighbours
    num_gfp_neighbours[0] = 0
    num_neighbours[0] = 0
    print(num_gfp_neighbours)
    #max_idx = np.amax(num_gfp_neighbours)
    #print(num_gfp_neighbours[max_idx])
    #print(np.mean(num_gfp_neighbours))
    #print(gfp_num_neighbours)
    #print(np.mean(gfp_num_neighbours))

    all_ind.append([np.argwhere(gfp >=0).flatten().astype(int)])
    gfp_ind.append([np.argwhere(gfp == 3).flatten().astype(int)])
    nogfp_ind.append([np.argwhere(gfp == 1).flatten().astype(int)])
    
    neighbour.append ([num_neighbours])
    gfp_neighbour.append([num_gfp_neighbours])
    
    #print(nogfp_ind)
    
    #print(gfp_neighbours)
    #print(np.mean)

    #plt.imshow(gfp_img, cmap='inferno')
    #plt.imshow(outlines_arr>0, cmap='Reds', alpha = 0.5)
    #plt.imshow(mask_posneg==3, cmap='gray', alpha=0.2)

In [ ]:
import plotly
import plotly.express as px
plotly.io.orca.config.executable = '/usr/local/bin/orca'
px.defaults.width = 2048
px.defaults.height = 2048

In [ ]:
import plotly.express as px
import pandas as pd
import numpy_indexed as npi
import statistics
import csv
import math

px = 0.2075665

all_props = []
props_list = []
nogfp_neighbour_areas = []
nogfp_noneighbour_areas = []
mean_nogfp_neighbour_areas = []
nogfp_noneighbour_ind_areas = []
mean_nogfp_noneighbour_areas = []


nogfp_neighbour_eccentricity = []

nogfp_noneighbour_eccentricity = []
mean_nogfp_neighbour_eccentricity = []

mean_nogfp_noneighbour_eccentricity = []

area_vs_neighbours = []

all_areas = []
all_eccentricities = []
all_mean_intensities = []
all_orientations = []
all_perimeters = []
all_aspect_ratios = []
all_shape_factors = []
all_solidities = []
all_zo1_intensities = []
all_circularities = []

all_gfp_areas = []
all_nogfp_areas = []
all_nogfp_no_neighbour_areas = []
all_nogfp_neighbour_areas = []

all_gfp_perimeters = []
all_nogfp_perimeters = []
all_nogfp_no_neighbour_perimeters = []
all_nogfp_neighbour_perimeters = []

all_gfp_orientations = []
all_nogfp_orientations = []
all_nogfp_no_neighbour_orientations = []
all_nogfp_neighbour_orientations = []

all_gfp_mean_intensities = []
all_nogfp_mean_intensities = []
all_nogfp_no_neighbour_mean_intensities = []
all_nogfp_neighbour_mean_intensities = []

all_gfp_eccentricities = []
all_nogfp_eccentricities = []
all_nogfp_no_neighbour_eccentricities = []
all_nogfp_neighbour_eccentricities = []

all_gfp_aspect_ratios = []
all_nogfp_aspect_ratios = []
all_nogfp_no_neighbour_aspect_ratios = []
all_nogfp_neighbour_aspect_ratios = []

all_gfp_shape_factors = []
all_nogfp_shape_factors = []
all_nogfp_no_neighbour_shape_factors = []
all_nogfp_neighbour_shape_factors = []

all_gfp_solidities = []
all_nogfp_solidities = []
all_nogfp_no_neighbour_solidities = []
all_nogfp_neighbour_solidities = []

all_gfp_zo1_intensities = []
all_nogfp_zo1_intensities = []
all_nogfp_no_neighbour_zo1_intensities = []
all_nogfp_neighbour_zo1_intensities = []

all_gfp_circularities = []
all_nogfp_circularities = []
all_nogfp_no_neighbour_circularities = []
all_nogfp_neighbour_circularities = []

average_all_areas = []
average_all_gfp_areas = []

all_areas_vs_gfp_neighbours = []
all_eccentricities_vs_gfp_neighbours = []
all_mean_intensities_vs_gfp_neighbours = []
all_perimeters_vs_gfp_neighbours = []
all_orientations_vs_gfp_neighbours = []
all_aspect_ratios_vs_gfp_neighbours = []
all_shape_factors_vs_gfp_neighbours = []
all_solidities_vs_gfp_neighbours = []
all_zo1_intensities_vs_gfp_neighbours = []
all_circularities_vs_gfp_neighbours = []

all_areas_vs_all_neighbours = []
all_eccentricities_vs_all_neighbours = []
all_mean_intensities_vs_all_neighbours = []
all_perimeters_vs_all_neighbours = []
all_orientations_vs_all_neighbours = []
all_aspect_ratios_vs_all_neighbours = []
all_shape_factors_vs_all_neighbours = []
all_solidities_vs_all_neighbours = []
all_zo1_intensities_vs_all_neighbours = []
all_circularities_vs_all_neighbours = []

all_gfp_neighbours = []
i = 0
for k in imgd:

    
#k = list(imgd.keys())[0]
    print(k)
    #print(imgd[k][0])
    
    maski = clear_border(imgd[k][3], buffer_size=30,bgval=0, in_place=False)
    gfp_img_unscaled = imgd[k][1]
    zo1_img_unscaled = imgd[k][2]

    # initialise outlines array
    outlines_arr = np.zeros(maski.shape)

    outlines = plot.outlines_list(maski)
    # unique outlines values

    y = 1
    #print(outlines)
    for o in outlines:
        for p in o:
            #print(p)
            outlines_arr[p[1],p[0]] = y
        y += 1
    
    #print(outlines_arr.shape)
    #print(zo1_img.shape)
    
    #rescale gfp channel to 0 - 1 float
    gfp_img = rescale_intensity(gfp_img_unscaled.astype(float),in_range='image')
    
    
    #zo1_img = rescale_intensity(zo1_img_unscaled.astype(float),in_range='image') #uncomment if you want to scale
    
    zo1_img = zo1_img_unscaled
    props = regionprops(maski.astype(int), gfp_img)
    props_zo1 = regionprops(outlines_arr.astype(int), zo1_img)
    #print(len(props_zo1))
    #print(len(props))
    areas = []
    eccentricities = []
    gfp_areas = []
    mean_intensities = []
    orientations = []
    perimeters = []
    shape_factors = []
    solidities = []
    aspect_ratios = []
    num_gfp_neighbours = gfp_neighbour[i]
    zo1_inensities = []
    circularities = []
    
    for j in range(len(np.unique(maski))-1):
        area = props[j].area*px*px
        eccentricity = props[j].eccentricity
        mean_intensity = props[j].mean_intensity
        orientation = props[j].orientation
        perimeter = props[j].perimeter*px
        shape_factor = perimeter/math.sqrt(area)
        solidity = props[j].solidity
        minor_axis_length = props[j].minor_axis_length
        major_axis_length = props[j].major_axis_length
        aspect_ratio = major_axis_length/minor_axis_length
        zo1_intensity = props_zo1[j].mean_intensity
        circularity = (4 * area * math.pi) / (perimeter**2)
        areas.append(area)
        eccentricities.append(eccentricity)
        mean_intensities.append(mean_intensity)
        orientations.append(orientation)
        perimeters.append(perimeter)
        shape_factors.append(shape_factor)
        solidities.append(solidity)
        aspect_ratios.append(aspect_ratio)
        zo1_inensities.append(zo1_intensity)
        circularities.append(circularity)
        
    print('max shape factor is ' + str(np.amax(shape_factors)) + ' at ' + str(np.where(shape_factors == np.amax(shape_factors))))   
    #print(gfp_ind[i][0])
    #print(gfp_ind[i][0]-1)
    gfp_areas = [areas[i] for i in (gfp_ind[i][0]-1)]
    gfp_eccentricities = [eccentricities[i] for i in (gfp_ind[i][0]-1)]
    gfp_mean_intensities = [mean_intensities[i] for i in (gfp_ind[i][0]-1)]
    gfp_orientations = [orientations[i] for i in (gfp_ind[i][0]-1)]
    gfp_perimeters = [perimeters[i] for i in (gfp_ind[i][0]-1)]
    gfp_shape_factors = [shape_factors[i] for i in (gfp_ind[i][0]-1)]
    gfp_solidities = [solidities[i] for i in (gfp_ind[i][0]-1)]
    gfp_aspect_ratios = [aspect_ratios[i] for i in (gfp_ind[i][0]-1)]
    gfp_zo1_intensities = [zo1_inensities[i] for i in (gfp_ind[i][0]-1)]
    gfp_circularities = [circularities[i] for i in (gfp_ind[i][0]-1)]
    
    no_gfp_areas = [areas[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_eccentricities = [eccentricities[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_mean_intensities = [mean_intensities[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_orientations = [orientations[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_perimeters = [perimeters[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_shape_factors = [shape_factors[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_solidities = [solidities[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_aspect_ratios = [aspect_ratios[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_zo1_intensities = [zo1_inensities[i] for i in (nogfp_ind[i][0]-1)]
    no_gfp_circularities = [circularities[i] for i in (nogfp_ind[i][0]-1)]
    
    no_gfp_no_neighbours_areas = [areas[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_eccentricities = [eccentricities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_mean_intensities = [mean_intensities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_orientations = [orientations[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_perimeters = [perimeters[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_shape_factors = [shape_factors[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_solidities = [solidities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_aspect_ratios = [aspect_ratios[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_zo1_intensities = [zo1_inensities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    no_gfp_no_neighbours_circularities = [circularities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] == 0]
    
    no_gfp_neighbours_areas = [areas[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_eccentricities = [eccentricities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_mean_intensities = [mean_intensities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >=1]
    no_gfp_neighbours_orientations = [orientations[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >=1]
    no_gfp_neighbours_perimeters = [perimeters[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_shape_factors = [shape_factors[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_solidities = [solidities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_aspect_ratios = [aspect_ratios[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_zo1_intensities = [zo1_inensities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    no_gfp_neighbours_circularities = [circularities[i] for i in (nogfp_ind[i][0]-1) if num_gfp_neighbours[0][i] >= 1]
    
    #print(no_gfp_no_neighbours_perimeters)
    
    #nogfp_ind_corr = nogfp_ind-1
    
    all_areas.append([areas])
    all_eccentricities.append(eccentricities)
    all_mean_intensities.append(mean_intensities)
    all_orientations.append(orientations)
    all_perimeters.append(perimeters)
    all_shape_factors.append(shape_factors)
    all_solidities.append(solidities)
    all_aspect_ratios.append(aspect_ratios)
    all_zo1_intensities.append(zo1_inensities)
    all_circularities.append(circularities)
    
    all_gfp_neighbours.append([neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)])
    
    areas_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    eccentricities_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    mean_intensities_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    orientations_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    perimeters_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    shape_factors_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    solidities_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    aspect_ratios_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    zo1_intensities_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    circularities_vs_gfp_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    
    areas_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)    
    eccentricities_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    mean_intensities_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    orientations_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    perimeters_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    shape_factors_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    solidities_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    aspect_ratios_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    zo1_intensities_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    circularities_vs_all_neighbours = np.empty([(len(no_gfp_areas)),2], dtype=float)
    
    areas_vs_gfp_neighbours[:,0] = no_gfp_areas
    eccentricities_vs_gfp_neighbours[:,0] = no_gfp_eccentricities
    mean_intensities_vs_gfp_neighbours[:,0] = no_gfp_mean_intensities
    orientations_vs_gfp_neighbours[:,0] = no_gfp_orientations
    perimeters_vs_gfp_neighbours[:,0] = no_gfp_perimeters
    shape_factors_vs_gfp_neighbours[:,0] = no_gfp_shape_factors
    solidities_vs_gfp_neighbours[:,0] = no_gfp_solidities
    aspect_ratios_vs_gfp_neighbours[:,0] = no_gfp_aspect_ratios
    zo1_intensities_vs_gfp_neighbours[:,0] = no_gfp_zo1_intensities
    circularities_vs_gfp_neighbours[:,0] = no_gfp_circularities
    
    areas_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    eccentricities_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    mean_intensities_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    orientations_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    perimeters_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    shape_factors_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]   
    solidities_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]   
    aspect_ratios_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]   
    zo1_intensities_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)] 
    circularities_vs_gfp_neighbours[:,1] = [gfp_neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)] 
    
    areas_vs_all_neighbours[:,0] = no_gfp_areas
    eccentricities_vs_all_neighbours[:,0] = no_gfp_eccentricities
    mean_intensities_vs_all_neighbours[:,0] = no_gfp_mean_intensities
    orientations_vs_all_neighbours[:,0] = no_gfp_orientations
    perimeters_vs_all_neighbours[:,0] = no_gfp_perimeters
    shape_factors_vs_all_neighbours[:,0] = no_gfp_shape_factors
    solidities_vs_all_neighbours[:,0] = no_gfp_solidities
    aspect_ratios_vs_all_neighbours[:,0] = no_gfp_aspect_ratios    
    zo1_intensities_vs_all_neighbours[:,0] = no_gfp_zo1_intensities
    circularities_vs_all_neighbours[:,0] = no_gfp_circularities
    
    areas_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    eccentricities_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    mean_intensities_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    orientations_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]
    perimeters_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    shape_factors_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    solidities_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    aspect_ratios_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    zo1_intensities_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    circularities_vs_all_neighbours[:,1] = [neighbour[i][0][j] for j in (nogfp_ind[i][0]-1)]    
    
    #print(areas_vs_gfp_neighbours.shape)
    
    df = pd.DataFrame(areas_vs_gfp_neighbours).groupby(1, as_index=True).mean()
    df_transposed = df.T
    
    #print(df_transposed)
    df_nei = pd.DataFrame(areas_vs_all_neighbours).groupby(1, as_index=True).mean()
    df_nei_transposed = df_nei.T
    
    ecc_gfp_df = pd.DataFrame(eccentricities_vs_gfp_neighbours).groupby(1, as_index=True).mean()
    ecc_gfp_df_transposed = df.T
    
    ecc_df = pd.DataFrame(areas_vs_all_neighbours).groupby(1, as_index=True).mean()
    ecc_df_transposed = df_nei.T
    
    if i == 0:
        df_gfp_all = df_transposed
        df_all = df_nei_transposed
        ecc_gfp_all = ecc_gfp_df_transposed
        ecc_all = ecc_df_transposed
    else:
        df_gfp_all = df_gfp_all.append(df_transposed)
        df_all = df_all.append(df_nei_transposed)
        ecc_gfp_all = df_gfp_all.append(ecc_gfp_df_transposed)
        ecc_all = df_all.append(ecc_df_transposed)        
    
    #print(df_transposed)
    #res = df[['index', 0]].values
    
    #areas_vs_gfp_neighbours = (np.array(zip(areas,gfp_neighbour[i][0][1:])))
    
    
    #avg_areas_vs_gfp_neighbours = npi.group_by(areas_vs_gfp_neighbours[:, 0]).split(areas_vs_gfp_neighbours[:, 1])
    
    #all_areas_vs_gfp_neighbours.append(areas_vs_gfp_neighbours)
    
    
    all_gfp_areas.append(gfp_areas)
    all_nogfp_areas.append(no_gfp_areas)
    all_nogfp_no_neighbour_areas.append(no_gfp_no_neighbours_areas)
    all_nogfp_neighbour_areas.append(no_gfp_neighbours_areas)
    average_all_areas.append(np.mean(areas))

    all_gfp_eccentricities.append(gfp_eccentricities)
    all_nogfp_eccentricities.append(no_gfp_eccentricities)
    all_nogfp_no_neighbour_eccentricities.append(no_gfp_no_neighbours_eccentricities)
    all_nogfp_neighbour_eccentricities.append(no_gfp_neighbours_eccentricities)
    
    all_gfp_mean_intensities.append(gfp_mean_intensities)
    all_nogfp_mean_intensities.append(no_gfp_mean_intensities)
    all_nogfp_no_neighbour_mean_intensities.append(no_gfp_no_neighbours_mean_intensities)
    all_nogfp_neighbour_mean_intensities.append(no_gfp_neighbours_mean_intensities)
    
    all_gfp_perimeters.append(gfp_perimeters)
    all_nogfp_perimeters.append(no_gfp_perimeters)
    all_nogfp_no_neighbour_perimeters.append(no_gfp_no_neighbours_perimeters)
    all_nogfp_neighbour_perimeters.append(no_gfp_neighbours_perimeters)
    
    all_gfp_orientations.append(gfp_orientations)
    all_nogfp_orientations.append(no_gfp_orientations)
    all_nogfp_no_neighbour_orientations.append(no_gfp_no_neighbours_orientations)    
    all_nogfp_neighbour_orientations.append(no_gfp_neighbours_orientations)  
    
    all_gfp_aspect_ratios.append(gfp_aspect_ratios)
    all_nogfp_aspect_ratios.append(no_gfp_aspect_ratios)
    all_nogfp_no_neighbour_aspect_ratios.append(no_gfp_no_neighbours_aspect_ratios) 
    all_nogfp_neighbour_aspect_ratios.append(no_gfp_neighbours_aspect_ratios) 
    
    all_gfp_shape_factors.append(gfp_shape_factors)
    all_nogfp_shape_factors.append(no_gfp_shape_factors)
    all_nogfp_no_neighbour_shape_factors.append(no_gfp_no_neighbours_shape_factors) 
    all_nogfp_neighbour_shape_factors.append(no_gfp_neighbours_shape_factors) 
    
    all_gfp_solidities.append(gfp_solidities)
    all_nogfp_solidities.append(no_gfp_solidities)
    all_nogfp_no_neighbour_solidities.append(no_gfp_no_neighbours_solidities)   
    all_nogfp_neighbour_solidities.append(no_gfp_neighbours_solidities)   

    all_gfp_zo1_intensities.append(gfp_zo1_intensities)
    all_nogfp_zo1_intensities.append(no_gfp_zo1_intensities)
    all_nogfp_no_neighbour_zo1_intensities.append(no_gfp_no_neighbours_zo1_intensities)   
    all_nogfp_neighbour_zo1_intensities.append(no_gfp_neighbours_zo1_intensities)   

    all_gfp_circularities.append(gfp_circularities)
    all_nogfp_circularities.append(no_gfp_circularities)
    all_nogfp_no_neighbour_circularities.append(no_gfp_no_neighbours_circularities)   
    all_nogfp_neighbour_circularities.append(no_gfp_neighbours_circularities)   
    
    all_areas_vs_gfp_neighbours.append(areas_vs_gfp_neighbours)
    all_eccentricities_vs_gfp_neighbours.append(eccentricities_vs_gfp_neighbours)
    all_mean_intensities_vs_gfp_neighbours.append(mean_intensities_vs_gfp_neighbours)
    all_perimeters_vs_gfp_neighbours.append(perimeters_vs_gfp_neighbours)
    all_orientations_vs_gfp_neighbours.append(orientations_vs_gfp_neighbours)
    all_aspect_ratios_vs_gfp_neighbours.append(aspect_ratios_vs_gfp_neighbours)
    all_shape_factors_vs_gfp_neighbours.append(shape_factors_vs_gfp_neighbours)
    all_solidities_vs_gfp_neighbours.append(solidities_vs_gfp_neighbours)
    all_zo1_intensities_vs_gfp_neighbours.append(zo1_intensities_vs_gfp_neighbours)
    all_circularities_vs_gfp_neighbours.append(circularities_vs_gfp_neighbours)
    
    all_areas_vs_all_neighbours.append(areas_vs_all_neighbours)
    all_eccentricities_vs_all_neighbours.append(eccentricities_vs_all_neighbours)
    all_mean_intensities_vs_all_neighbours.append(mean_intensities_vs_all_neighbours)
    all_perimeters_vs_all_neighbours.append(perimeters_vs_all_neighbours)
    all_orientations_vs_all_neighbours.append(orientations_vs_all_neighbours)
    all_aspect_ratios_vs_all_neighbours.append(aspect_ratios_vs_all_neighbours)
    all_shape_factors_vs_all_neighbours.append(shape_factors_vs_all_neighbours)
    all_solidities_vs_all_neighbours.append(solidities_vs_all_neighbours)
    all_zo1_intensities_vs_all_neighbours.append(zo1_intensities_vs_all_neighbours)
    all_circularities_vs_all_neighbours.append(circularities_vs_all_neighbours)
    
    i +=1


In [ ]:
#df_gfp_areas = pd.DataFrame(data = [val for sublist in all_gfp_areas for val in sublist], columns = ['eEGFP'])
from scipy.stats.mstats import winsorize
import pandas as pd
##areas
 
df_gfp_areas = pd.DataFrame(data = all_gfp_areas, index = [files,cre]).transpose()
df_nogfp_areas = pd.DataFrame(data = all_nogfp_areas, index = [files,cre]).transpose()
df_nogfp_no_neighbour_areas = pd.DataFrame(data = all_nogfp_no_neighbour_areas, index = [files,cre]).transpose()
df_nogfp_neighbour_areas = pd.DataFrame(data = all_nogfp_neighbour_areas, index = [files,cre]).transpose()

with pd.ExcelWriter('all_areas.xlsx') as writer:
    df_gfp_areas.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_areas.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_areas.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_areas = pd.melt(df_gfp_areas)
df_gfp_areas.columns = ['file', 'Genotype', 'Area']
df_gfp_areas.set_index(['file','Genotype'])
df_gfp_areas = df_gfp_areas.assign(GFP='eGFP')

df_gfp_areas_pv = df_gfp_areas.pivot(index=None, columns = 'Genotype', values = 'Area')
df_gfp_areas = df_gfp_areas[df_gfp_areas.Genotype != 'NoCre']

df_nogfp_areas = pd.melt(df_nogfp_no_neighbour_areas)
df_nogfp_areas.columns = ['file', 'Genotype', 'Area']
df_nogfp_areas.set_index(['file','Genotype'])
df_nogfp_areas = df_nogfp_areas.assign(GFP='tdTom (D)')

df_nogfp_neighbour_areas = pd.melt(df_nogfp_neighbour_areas)
df_nogfp_neighbour_areas.columns = ['file', 'Genotype', 'Area']
df_nogfp_neighbour_areas.set_index(['file','Genotype'])
df_nogfp_neighbour_areas = df_nogfp_neighbour_areas.assign(GFP='tdTom (N)')

#df_nogfp_areas_pv = df_nogfp_areas.pivot(index=None, columns = 'Genotype', values = 'Area')

NoCre_nogfp_areas = df_nogfp_areas[df_nogfp_areas.Genotype == 'NoCre']
NoCre_nogfp_areas = NoCre_nogfp_areas.assign(GFP = 'tdTom')

df_nogfp_areas = df_nogfp_areas[df_nogfp_areas.Genotype != 'NoCre']
df_nogfp_neighbour_areas = df_nogfp_neighbour_areas[df_nogfp_neighbour_areas.Genotype != 'NoCre']

#df_nogfp_aspect_ratios_pv.groupby('file')
df_areas_grouped_nocre = NoCre_nogfp_areas.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_areas_grouped_nogfp = df_nogfp_areas.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_areas_grouped_nogfp_neighbours = df_nogfp_neighbour_areas.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_areas_grouped_gfp = df_gfp_areas.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
pd.options.display.max_colwidth = 150
display(df_areas_grouped_gfp)
df_areas_grouped_join = df_areas_grouped_nogfp.copy()
df_areas_grouped_join['GFP2'] = df_areas_grouped_gfp["GFP"]
df_areas_grouped_join['GFP2_area'] = df_areas_grouped_gfp["Area"]
df_areas_grouped_join['GFP3'] = df_areas_grouped_nogfp_neighbours["GFP"]
df_areas_grouped_join['GFP3_area'] = df_areas_grouped_nogfp_neighbours["Area"]

print(df_areas_grouped_join)

df_areas_grouped_join.to_excel('mean_areas_groupby_GFP_tdtom_ND.xlsx')


## circularitiy
 
df_gfp_circularities = pd.DataFrame(data = all_gfp_circularities, index = [files,cre]).transpose()
df_nogfp_circularities = pd.DataFrame(data = all_nogfp_circularities, index = [files,cre]).transpose()
df_nogfp_no_neighbour_circularities = pd.DataFrame(data = all_nogfp_no_neighbour_circularities, index = [files,cre]).transpose()
df_nogfp_neighbour_circularities = pd.DataFrame(data = all_nogfp_neighbour_circularities, index = [files,cre]).transpose()

with pd.ExcelWriter('all_circularities.xlsx') as writer:
    df_gfp_circularities.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_circularities.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_circularities.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_circularities = pd.melt(df_gfp_circularities)
df_gfp_circularities.columns = ['file', 'Genotype', 'Circularity']
df_gfp_circularities.set_index(['file','Genotype'])
df_gfp_circularities = df_gfp_circularities.assign(GFP='eGFP')

df_gfp_circularities_pv = df_gfp_circularities.pivot(index=None, columns = 'Genotype', values = 'Circularity')
df_gfp_circularities = df_gfp_circularities[df_gfp_circularities.Genotype != 'NoCre']

df_nogfp_circularities = pd.melt(df_nogfp_no_neighbour_circularities)
df_nogfp_circularities.columns = ['file', 'Genotype', 'Circularity']
df_nogfp_circularities.set_index(['file','Genotype'])
df_nogfp_circularities = df_nogfp_circularities.assign(GFP='tdTom (D)')

df_nogfp_neighbour_circularities = pd.melt(df_nogfp_neighbour_circularities)
df_nogfp_neighbour_circularities.columns = ['file', 'Genotype', 'Circularity']
df_nogfp_neighbour_circularities.set_index(['file','Genotype'])
df_nogfp_neighbour_circularities = df_nogfp_neighbour_circularities.assign(GFP='tdTom (N)')

#df_nogfp_circularities_pv = df_nogfp_circularities.pivot(index=None, columns = 'Genotype', values = 'Circularity')

NoCre_nogfp_circularities = df_nogfp_circularities[df_nogfp_circularities.Genotype == 'NoCre']
NoCre_nogfp_circularities = NoCre_nogfp_circularities.assign(GFP = 'tdTom')

df_nogfp_circularities = df_nogfp_circularities[df_nogfp_circularities.Genotype != 'NoCre']
df_nogfp_neighbour_circularities = df_nogfp_neighbour_circularities[df_nogfp_neighbour_circularities.Genotype != 'NoCre']

#df_nogfp_aspect_ratios_pv.groupby('file')
df_circularities_grouped_nocre = NoCre_nogfp_circularities.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_circularities_grouped_nogfp = df_nogfp_circularities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_circularities_grouped_nogfp_neighbours = df_nogfp_neighbour_circularities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_circularities_grouped_gfp = df_gfp_circularities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
pd.options.display.max_colwidth = 150
display(df_circularities_grouped_gfp)
df_circularities_grouped_join = df_circularities_grouped_nogfp.copy()
df_circularities_grouped_join['GFP2'] = df_circularities_grouped_gfp["GFP"]
df_circularities_grouped_join['GFP2_circularity'] = df_circularities_grouped_gfp["Circularity"]
df_circularities_grouped_join['GFP3'] = df_circularities_grouped_nogfp_neighbours["GFP"]
df_circularities_grouped_join['GFP3_circularity'] = df_circularities_grouped_nogfp_neighbours["Circularity"]

print(df_circularities_grouped_join)

df_circularities_grouped_join.to_excel('mean_circularities_groupby_GFP_tdtom_ND.xlsx')


##eccentricities

df_gfp_eccentricities = pd.DataFrame(data = all_gfp_eccentricities, index = [files,cre]).transpose()
df_nogfp_eccentricities = pd.DataFrame(data = all_nogfp_eccentricities, index = [files,cre]).transpose()
df_nogfp_no_neighbour_eccentricities = pd.DataFrame(data = all_nogfp_no_neighbour_eccentricities, index = [files,cre]).transpose()
df_nogfp_neighbour_eccentricities = pd.DataFrame(data = all_nogfp_neighbour_eccentricities, index = [files,cre]).transpose()

with pd.ExcelWriter('all_eccentricities.xlsx') as writer:
    df_gfp_eccentricities.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_eccentricities.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_eccentricities.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_eccentricities = pd.melt(df_gfp_eccentricities)
df_gfp_eccentricities.columns = ['file', 'Genotype', 'Eccentricity']
df_gfp_eccentricities.set_index(['file','Genotype'])
df_gfp_eccentricities = df_gfp_eccentricities.assign(GFP='EGFP')

df_gfp_eccentricities_pv = df_gfp_eccentricities.pivot(index=None, columns = 'Genotype', values = 'Eccentricity')
df_gfp_eccentricities = df_gfp_eccentricities[df_gfp_eccentricities.Genotype != 'NoCre']


df_nogfp_eccentricities = pd.melt(df_nogfp_no_neighbour_eccentricities)
df_nogfp_eccentricities.columns = ['file', 'Genotype', 'Eccentricity']
df_nogfp_eccentricities.set_index(['file','Genotype'])
df_nogfp_eccentricities = df_nogfp_eccentricities.assign(GFP='tdTom (D)')
df_nogfp_eccentricities_pv = df_nogfp_eccentricities.pivot(index=None, columns = 'Genotype', values = 'Eccentricity')

df_nogfp_neighbour_eccentricities = pd.melt(df_nogfp_neighbour_eccentricities)
df_nogfp_neighbour_eccentricities.columns = ['file', 'Genotype', 'Eccentricity']
df_nogfp_neighbour_eccentricities.set_index(['file','Genotype'])
df_nogfp_neighbour_eccentricities = df_nogfp_neighbour_eccentricities.assign(GFP='tdTom (N)')

NoCre_nogfp_eccentricities = df_nogfp_eccentricities[df_nogfp_eccentricities.Genotype == 'NoCre']
df_nogfp_eccentricities = df_nogfp_eccentricities[df_nogfp_eccentricities.Genotype != 'NoCre']
df_nogfp_neighbour_eccentricities = df_nogfp_neighbour_eccentricities[df_nogfp_neighbour_eccentricities.Genotype != 'NoCre']

#df_nogfp_aspect_ratios_pv.groupby('file')
df_eccentricities_grouped_nocre = NoCre_nogfp_eccentricities.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_eccentricities_grouped_nogfp = df_nogfp_eccentricities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_eccentricities_grouped_gfp = df_gfp_eccentricities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_eccentricities_grouped_nogfp_neighbours = df_nogfp_neighbour_eccentricities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_eccentricities_grouped_join = df_eccentricities_grouped_nogfp.copy()
df_eccentricities_grouped_join['GFP2'] = df_eccentricities_grouped_gfp["GFP"]
df_eccentricities_grouped_join['GFP_eccentricity'] = df_eccentricities_grouped_gfp["Eccentricity"]
df_eccentricities_grouped_join['GFP3'] = df_eccentricities_grouped_nogfp_neighbours["GFP"]
df_eccentricities_grouped_join['GFP3_eccentricity'] = df_eccentricities_grouped_nogfp_neighbours["Eccentricity"]

df_eccentricities_grouped_join.to_excel('mean_eccentricities_groupby_GFP_tdtom_ND.xlsx')



##shape_factors

df_gfp_shape_factors = pd.DataFrame(data = all_gfp_shape_factors, index = [files,cre]).transpose()
df_nogfp_shape_factors = pd.DataFrame(data = all_nogfp_shape_factors, index = [files,cre]).transpose()
df_nogfp_no_neighbour_shape_factors = pd.DataFrame(data = all_nogfp_no_neighbour_shape_factors, index = [files,cre]).transpose()
df_nogfp_neighbour_shape_factors = pd.DataFrame(data = all_nogfp_neighbour_shape_factors, index = [files,cre]).transpose()

with pd.ExcelWriter('all_shape_factors.xlsx') as writer:
    df_gfp_shape_factors.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_shape_factors.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_shape_factors.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_shape_factors = pd.melt(df_gfp_shape_factors)
df_gfp_shape_factors.columns = ['file', 'Genotype', 'Shape Factor']
df_gfp_shape_factors.set_index(['file','Genotype'])
df_gfp_shape_factors = df_gfp_shape_factors.assign(GFP='EGFP')

df_gfp_shape_factors_pv = df_gfp_shape_factors.pivot(index=None, columns = 'Genotype', values = 'Shape Factor')
df_gfp_shape_factors = df_gfp_shape_factors[df_gfp_shape_factors.Genotype != 'NoCre']


df_nogfp_shape_factors = pd.melt(df_nogfp_no_neighbour_shape_factors)
df_nogfp_shape_factors.columns = ['file', 'Genotype', 'Shape Factor']
df_nogfp_shape_factors.set_index(['file','Genotype'])
df_nogfp_shape_factors = df_nogfp_shape_factors.assign(GFP='tdTom (D)')
df_nogfp_shape_factors_pv = df_nogfp_shape_factors.pivot(index=None, columns = 'Genotype', values = 'Shape Factor')

df_nogfp_neighbour_shape_factors = pd.melt(df_nogfp_neighbour_shape_factors)
df_nogfp_neighbour_shape_factors.columns = ['file', 'Genotype', 'Shape Factor']
df_nogfp_neighbour_shape_factors.set_index(['file','Genotype'])
df_nogfp_neighbour_shape_factors = df_nogfp_neighbour_shape_factors.assign(GFP='tdTom (N)')

NoCre_nogfp_shape_factors = df_nogfp_shape_factors[df_nogfp_shape_factors.Genotype == 'NoCre']
NoCre_nogfp_shape_factors = NoCre_nogfp_shape_factors.assign(GFP = 'tdTom')

df_nogfp_shape_factors = df_nogfp_shape_factors[df_nogfp_shape_factors.Genotype != 'NoCre']
df_nogfp_neighbour_shape_factors = df_nogfp_neighbour_shape_factors[df_nogfp_neighbour_shape_factors.Genotype != 'NoCre']


#df_nogfp_aspect_ratios_pv.groupby('file')
df_shape_factors_grouped_nocre = NoCre_nogfp_shape_factors.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_shape_factors_grouped_nogfp = df_nogfp_shape_factors.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_shape_factors_grouped_gfp = df_gfp_shape_factors.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_shape_factors_grouped_nogfp_neighbours = df_nogfp_neighbour_shape_factors.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_shape_factors_grouped_join = df_shape_factors_grouped_nogfp.copy()
df_shape_factors_grouped_join['GFP2'] = df_shape_factors_grouped_gfp["GFP"]
df_shape_factors_grouped_join['GFP2_shapefactor'] = df_shape_factors_grouped_gfp["Shape Factor"]
df_shape_factors_grouped_join['GFP3'] = df_shape_factors_grouped_nogfp_neighbours["GFP"]
df_shape_factors_grouped_join['GFP3_shapefactor'] = df_shape_factors_grouped_nogfp_neighbours["Shape Factor"]

df_shape_factors_grouped_join.to_excel('Shape_factors_groupby_GFP_tdtom_ND.xlsx')

##aspect_ratios

df_gfp_aspect_ratios = pd.DataFrame(data = all_gfp_aspect_ratios, index = [files,cre]).transpose()
df_nogfp_aspect_ratios = pd.DataFrame(data = all_nogfp_aspect_ratios, index = [files,cre]).transpose()
df_nogfp_no_neighbour_aspect_ratios = pd.DataFrame(data = all_nogfp_no_neighbour_aspect_ratios, index = [files,cre]).transpose()
df_nogfp_neighbour_aspect_ratios = pd.DataFrame(data = all_nogfp_neighbour_aspect_ratios, index = [files,cre]).transpose()

with pd.ExcelWriter('all_aspect_ratios.xlsx') as writer:
    df_gfp_aspect_ratios.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_aspect_ratios.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_aspect_ratios.to_excel(writer, sheet_name='tdTom (D) cells')


df_gfp_aspect_ratios = pd.melt(df_gfp_aspect_ratios)
df_gfp_aspect_ratios.columns = ['file', 'Genotype', 'Aspect Ratio']
df_gfp_aspect_ratios.set_index(['file','Genotype'])
df_gfp_aspect_ratios = df_gfp_aspect_ratios.assign(GFP='EGFP')

df_gfp_aspect_ratios_pv = df_gfp_aspect_ratios.pivot(index=None, columns = 'Genotype', values = 'Aspect Ratio')

df_gfp_aspect_ratios = df_gfp_aspect_ratios[df_gfp_aspect_ratios.Genotype != 'NoCre']


df_nogfp_aspect_ratios = pd.melt(df_nogfp_no_neighbour_aspect_ratios)
df_nogfp_aspect_ratios.columns = ['file', 'Genotype', 'Aspect Ratio']
df_nogfp_aspect_ratios.set_index(['file','Genotype'])
df_nogfp_aspect_ratios = df_nogfp_aspect_ratios.assign(GFP='tdTom (D)')
df_nogfp_aspect_ratios_pv = df_nogfp_aspect_ratios.pivot(index=None, columns = 'Genotype', values = 'Aspect Ratio')

df_nogfp_neighbour_aspect_ratios = pd.melt(df_nogfp_neighbour_aspect_ratios)
df_nogfp_neighbour_aspect_ratios.columns = ['file', 'Genotype', 'Aspect Ratio']
df_nogfp_neighbour_aspect_ratios.set_index(['file','Genotype'])
df_nogfp_neighbour_aspect_ratios = df_nogfp_neighbour_aspect_ratios.assign(GFP='tdTom (N)')

NoCre_nogfp_aspect_ratios = df_nogfp_aspect_ratios[df_nogfp_aspect_ratios.Genotype == 'NoCre']
NoCre_nogfp_aspect_ratios = NoCre_nogfp_aspect_ratios.assign(GFP = 'tdTom')

df_nogfp_aspect_ratios = df_nogfp_aspect_ratios[df_nogfp_aspect_ratios.Genotype != 'NoCre']
df_nogfp_neighbour_aspect_ratios = df_nogfp_neighbour_aspect_ratios[df_nogfp_neighbour_aspect_ratios.Genotype != 'NoCre']

#df_nogfp_aspect_ratios_pv.groupby('file')

df_aspect_ratios_grouped_nocre = NoCre_nogfp_aspect_ratios.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_aspect_ratios_grouped_nogfp = df_nogfp_aspect_ratios.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_aspect_ratios_grouped_gfp = df_gfp_aspect_ratios.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_aspect_ratios_grouped_nogfp_neighbours = df_nogfp_neighbour_aspect_ratios.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_aspect_ratios_grouped_join = df_aspect_ratios_grouped_nogfp.copy()
df_aspect_ratios_grouped_join['GFP2'] = df_aspect_ratios_grouped_gfp["GFP"]
df_aspect_ratios_grouped_join['GFP_aspect_ratio'] = df_aspect_ratios_grouped_gfp["Aspect Ratio"]
df_aspect_ratios_grouped_join['GFP2_aspectratio'] = df_aspect_ratios_grouped_gfp["Aspect Ratio"]
df_aspect_ratios_grouped_join['GFP3'] = df_aspect_ratios_grouped_nogfp_neighbours["GFP"]
df_aspect_ratios_grouped_join['GFP3_aspectratio'] = df_aspect_ratios_grouped_nogfp_neighbours["Aspect Ratio"]

df_aspect_ratios_grouped_join.to_excel('mean_aspect_ratios_groupby_GFP_tdtom_ND.xlsx')

##mean intensities

df_gfp_mean_intensities = pd.DataFrame(data = all_gfp_mean_intensities, index = [files,cre]).transpose()
df_nogfp_mean_intensities = pd.DataFrame(data = all_nogfp_mean_intensities, index = [files,cre]).transpose()
df_nogfp_no_neighbour_mean_intensities = pd.DataFrame(data = all_nogfp_no_neighbour_mean_intensities, index = [files,cre]).transpose()
df_nogfp_neighbour_mean_intensities = pd.DataFrame(data = all_nogfp_neighbour_mean_intensities, index = [files,cre]).transpose()

with pd.ExcelWriter('all_mean_EGFP_intensities.xlsx') as writer:
    df_gfp_mean_intensities.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_mean_intensities.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_mean_intensities.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_mean_intensities = pd.melt(df_gfp_mean_intensities)
df_gfp_mean_intensities.columns = ['file', 'Genotype', 'Mean Intensity']
df_gfp_mean_intensities.set_index(['file','Genotype'])
df_gfp_mean_intensities = df_gfp_mean_intensities.assign(GFP='EGFP')

df_gfp_mean_intensities_pv = df_gfp_mean_intensities.pivot(index=None, columns = 'Genotype', values = 'Mean Intensity')

df_gfp_mean_intensities = df_gfp_mean_intensities[df_gfp_mean_intensities.Genotype != 'NoCre']


df_nogfp_mean_intensities = pd.melt(df_nogfp_no_neighbour_mean_intensities)
df_nogfp_mean_intensities.columns = ['file', 'Genotype', 'Mean Intensity']
df_nogfp_mean_intensities.set_index(['file','Genotype'])
df_nogfp_mean_intensities = df_nogfp_mean_intensities.assign(GFP='tdTom (D)')
df_nogfp_mean_intensities_pv = df_nogfp_mean_intensities.pivot(index=None, columns = 'Genotype', values = 'Mean Intensity')

df_nogfp_neighbour_mean_intensities = pd.melt(df_nogfp_neighbour_mean_intensities)
df_nogfp_neighbour_mean_intensities.columns = ['file', 'Genotype', 'Mean Intensity']
df_nogfp_neighbour_mean_intensities.set_index(['file','Genotype'])
df_nogfp_neighbour_mean_intensities = df_nogfp_neighbour_mean_intensities.assign(GFP='tdTom (N)')

NoCre_nogfp_mean_intensities = df_nogfp_mean_intensities[df_nogfp_mean_intensities.Genotype == 'NoCre']
NoCre_nogfp_mean_intensities = NoCre_nogfp_mean_intensities.assign(GFP = 'tdTom')

df_nogfp_mean_intensities = df_nogfp_mean_intensities[df_nogfp_mean_intensities.Genotype != 'NoCre']
df_nogfp_neighbour_mean_intensities = df_nogfp_neighbour_mean_intensities[df_nogfp_neighbour_mean_intensities.Genotype != 'NoCre']

#df_nogfp_mean_intensities_pv.groupby('file')

df_mean_intensities_grouped_nocre = NoCre_nogfp_mean_intensities.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_mean_intensities_grouped_nogfp = df_nogfp_mean_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_mean_intensities_grouped_gfp = df_gfp_mean_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_mean_intensities_grouped_nogfp_neighbours = df_nogfp_neighbour_mean_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_mean_intensities_grouped_join = df_mean_intensities_grouped_nogfp.copy()
df_mean_intensities_grouped_join['GFP2'] = df_mean_intensities_grouped_gfp["GFP"]
df_mean_intensities_grouped_join['GFP_mean_intensity'] = df_mean_intensities_grouped_gfp["Mean Intensity"]
df_mean_intensities_grouped_join['GFP2_mean_intensity'] = df_mean_intensities_grouped_gfp["Mean Intensity"]
df_mean_intensities_grouped_join['GFP3'] = df_mean_intensities_grouped_nogfp_neighbours["GFP"]
df_mean_intensities_grouped_join['GFP3_mean_intensity'] = df_mean_intensities_grouped_nogfp_neighbours["Mean Intensity"]

df_mean_intensities_grouped_join.to_excel('mean_EGFP_intensity_groupby_GFP_tdtom_ND.xlsx')

##zo1_intensities

df_gfp_zo1_intensities = pd.DataFrame(data = all_gfp_zo1_intensities, index = [files,cre]).transpose()
df_nogfp_zo1_intensities = pd.DataFrame(data = all_nogfp_zo1_intensities, index = [files,cre]).transpose()
df_nogfp_no_neighbour_zo1_intensities = pd.DataFrame(data = all_nogfp_no_neighbour_zo1_intensities, index = [files,cre]).transpose()
df_nogfp_neighbour_zo1_intensities = pd.DataFrame(data = all_nogfp_neighbour_zo1_intensities, index = [files,cre]).transpose()

with pd.ExcelWriter('all_zo1_intensities.xlsx') as writer:
    df_gfp_zo1_intensities.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_zo1_intensities.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_zo1_intensities.to_excel(writer, sheet_name='tdTom (D) cells')


df_gfp_zo1_intensities = pd.melt(df_gfp_zo1_intensities)
df_gfp_zo1_intensities.columns = ['file', 'Genotype', 'Zo1 Intensity']
df_gfp_zo1_intensities.set_index(['file','Genotype'])
df_gfp_zo1_intensities = df_gfp_zo1_intensities.assign(GFP='EGFP')

df_gfp_zo1_intensities_pv = df_gfp_zo1_intensities.pivot(index=None, columns = 'Genotype', values = 'Zo1 Intensity')

df_gfp_zo1_intensities = df_gfp_zo1_intensities[df_gfp_zo1_intensities.Genotype != 'NoCre']


df_nogfp_zo1_intensities = pd.melt(df_nogfp_no_neighbour_zo1_intensities)
df_nogfp_zo1_intensities.columns = ['file', 'Genotype', 'Zo1 Intensity']
df_nogfp_zo1_intensities.set_index(['file','Genotype'])
df_nogfp_zo1_intensities = df_nogfp_zo1_intensities.assign(GFP='tdTom (D)')
df_nogfp_zo1_intensities_pv = df_nogfp_zo1_intensities.pivot(index=None, columns = 'Genotype', values = 'Zo1 Intensity')

df_nogfp_neighbour_zo1_intensities = pd.melt(df_nogfp_neighbour_zo1_intensities)
df_nogfp_neighbour_zo1_intensities.columns = ['file', 'Genotype', 'Zo1 Intensity']
df_nogfp_neighbour_zo1_intensities.set_index(['file','Genotype'])
df_nogfp_neighbour_zo1_intensities = df_nogfp_neighbour_zo1_intensities.assign(GFP='tdTom (N)')

NoCre_nogfp_zo1_intensities = df_nogfp_zo1_intensities[df_nogfp_zo1_intensities.Genotype == 'NoCre']
NoCre_nogfp_zo1_intensities = NoCre_nogfp_zo1_intensities.assign(GFP = 'tdTom')

df_nogfp_zo1_intensities = df_nogfp_zo1_intensities[df_nogfp_zo1_intensities.Genotype != 'NoCre']
df_nogfp_neighbour_zo1_intensities = df_nogfp_neighbour_zo1_intensities[df_nogfp_neighbour_zo1_intensities.Genotype != 'NoCre']

df_zo1_intensities_grouped_nocre = NoCre_nogfp_zo1_intensities.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_zo1_intensities_grouped_nogfp = df_nogfp_zo1_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_zo1_intensities_grouped_gfp = df_gfp_zo1_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_zo1_intensities_grouped_nogfp_neighbours = df_nogfp_neighbour_zo1_intensities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_zo1_intensities_grouped_join = df_zo1_intensities_grouped_nogfp.copy()
df_zo1_intensities_grouped_join['GFP2'] = df_zo1_intensities_grouped_gfp["GFP"]
df_zo1_intensities_grouped_join['GFP_zo1_intensity'] = df_zo1_intensities_grouped_gfp["Zo1 Intensity"]
df_zo1_intensities_grouped_join['GFP2_zo1_intensity'] = df_zo1_intensities_grouped_gfp["Zo1 Intensity"]
df_zo1_intensities_grouped_join['GFP3'] = df_zo1_intensities_grouped_nogfp_neighbours["GFP"]
df_zo1_intensities_grouped_join['GFP3_zo1_intensity'] = df_zo1_intensities_grouped_nogfp_neighbours["Zo1 Intensity"]

df_zo1_intensities_grouped_join.to_excel('mean_Zo1_intensity_groupby_GFP_tdtom_ND.xlsx')

##solidities

df_gfp_solidities = pd.DataFrame(data = all_gfp_solidities, index = [files,cre]).transpose()
df_nogfp_solidities = pd.DataFrame(data = all_nogfp_solidities, index = [files,cre]).transpose()
df_nogfp_no_neighbour_solidities = pd.DataFrame(data = all_nogfp_no_neighbour_solidities, index = [files,cre]).transpose()
df_nogfp_neighbour_solidities = pd.DataFrame(data = all_nogfp_neighbour_solidities, index = [files,cre]).transpose()

with pd.ExcelWriter('all_solidities.xlsx') as writer:
    df_gfp_solidities.to_excel(writer, sheet_name='EGFP cells')
    df_nogfp_neighbour_solidities.to_excel(writer, sheet_name='tdTom (N) cells')
    df_nogfp_no_neighbour_solidities.to_excel(writer, sheet_name='tdTom (D) cells')

df_gfp_solidities = pd.melt(df_gfp_solidities)
df_gfp_solidities.columns = ['file', 'Genotype', 'Solidity']
df_gfp_solidities.set_index(['file','Genotype'])
df_gfp_solidities = df_gfp_solidities.assign(GFP='EGFP')

df_gfp_solidities_pv = df_gfp_solidities.pivot(index=None, columns = 'Genotype', values = 'Solidity')

df_gfp_solidities = df_gfp_solidities[df_gfp_solidities.Genotype != 'NoCre']


df_nogfp_solidities = pd.melt(df_nogfp_no_neighbour_solidities)
df_nogfp_solidities.columns = ['file', 'Genotype', 'Solidity']
df_nogfp_solidities.set_index(['file','Genotype'])
df_nogfp_solidities = df_nogfp_solidities.assign(GFP='tdTom (D)')
df_nogfp_solidities_pv = df_nogfp_solidities.pivot(index=None, columns = 'Genotype', values = 'Solidity')

df_nogfp_neighbour_solidities = pd.melt(df_nogfp_neighbour_solidities)
df_nogfp_neighbour_solidities.columns = ['file', 'Genotype', 'Solidity']
df_nogfp_neighbour_solidities.set_index(['file','Genotype'])
df_nogfp_neighbour_solidities = df_nogfp_neighbour_solidities.assign(GFP='tdTom (N)')

NoCre_nogfp_solidities = df_nogfp_solidities[df_nogfp_solidities.Genotype == 'NoCre']
NoCre_nogfp_solidities = NoCre_nogfp_solidities.assign(GFP = 'tdTom')

df_nogfp_solidities = df_nogfp_solidities[df_nogfp_solidities.Genotype != 'NoCre']
df_nogfp_neighbour_solidities = df_nogfp_neighbour_solidities[df_nogfp_neighbour_solidities.Genotype != 'NoCre']

#df_nogfp_mean_intensities_pv.groupby('file')

df_solidities_grouped_nocre = NoCre_nogfp_solidities.groupby(['file','Genotype','GFP'], as_index=False).mean()
df_solidities_grouped_nogfp = df_nogfp_solidities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_solidities_grouped_gfp = df_gfp_solidities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]
df_solidities_grouped_nogfp_neighbours = df_nogfp_neighbour_solidities.groupby(['file', 'Genotype', 'GFP'], as_index=False).mean().iloc[[3,0,1,2,4,5,6,7,8]]

df_solidities_grouped_join = df_solidities_grouped_nogfp.copy()
df_solidities_grouped_join['GFP2'] = df_solidities_grouped_gfp["GFP"]
df_solidities_grouped_join['GFP_solidity'] = df_solidities_grouped_gfp["Solidity"]
df_solidities_grouped_join['GFP2_solidity'] = df_solidities_grouped_gfp["Solidity"]
df_solidities_grouped_join['GFP3'] = df_solidities_grouped_nogfp_neighbours["GFP"]
df_solidities_grouped_join['GFP3_solidity'] = df_solidities_grouped_nogfp_neighbours["Solidity"]

df_solidities_grouped_join.to_excel('mean_Solidities_groupby_GFP_tdtom_ND.xlsx')

In [ ]:
#plot results as each embryo, grouped according to gfp vs td tom and genotype

import plotly.graph_objects as go
import numpy as np

###
### areas
###

fig = go.Figure(data=go.Scatter(
    x = [df_areas_grouped_nocre['Genotype'],df_areas_grouped_nocre['GFP']],
    y = df_areas_grouped_nocre['Area'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_areas_grouped_gfp['Genotype'],df_areas_grouped_gfp['GFP']],
    y = df_areas_grouped_gfp['Area'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_areas_grouped_nogfp_neighbours['Genotype'],df_areas_grouped_nogfp_neighbours['GFP']],
    y = df_areas_grouped_nogfp_neighbours['Area'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_areas_grouped_nogfp['Genotype'],df_areas_grouped_nogfp['GFP']],
    y = df_areas_grouped_nogfp['Area'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_areas_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_area'], row['GFP2_area']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_area'], row['Area']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Area (\u03BCm\u00B2)',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)
from scipy.stats import f_oneway
#f_oneway(tillamook, newport, petersburg, magadan, tvarminne)

from statsmodels.sandbox.stats.multicomp import multipletests
#p_adjusted = multipletests(____, alpha=____, method='bonferroni')

#fl_stats = stats.ttest_ind(df_gfp_areas_pv['Cre;Fl/Fl'].dropna().values, df_nogfp_areas_pv['Cre;Fl/Fl'].dropna().values, equal_var = True)
#print(fl_stats)

#flnull_stats = stats.ttest_ind(df_gfp_areas_pv['Cre;Fl/-'].dropna().values, df_nogfp_areas_pv['Cre;Fl/-'].dropna().values, equal_var = True)
#print(flnull_stats)

fig.write_image("Areas.svg")

###
### circularity
###

fig = go.Figure(data=go.Scatter(
    x = [df_circularities_grouped_nocre['Genotype'],df_circularities_grouped_nocre['GFP']],
    y = df_circularities_grouped_nocre['Circularity'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_circularities_grouped_gfp['Genotype'],df_circularities_grouped_gfp['GFP']],
    y = df_circularities_grouped_gfp['Circularity'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_circularities_grouped_nogfp_neighbours['Genotype'],df_circularities_grouped_nogfp_neighbours['GFP']],
    y = df_circularities_grouped_nogfp_neighbours['Circularity'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_circularities_grouped_nogfp['Genotype'],df_circularities_grouped_nogfp['GFP']],
    y = df_circularities_grouped_nogfp['Circularity'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_circularities_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_circularity'], row['GFP2_circularity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_circularity'], row['Circularity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Circularity',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Circularities.svg")


###
### shape factors
###


print(df_shape_factors_grouped_nocre)
fig = go.Figure(data=go.Scatter(
    x = [df_shape_factors_grouped_nocre['Genotype'],df_shape_factors_grouped_nocre['GFP']],
    y = df_shape_factors_grouped_nocre['Shape Factor'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_shape_factors_grouped_gfp['Genotype'],df_shape_factors_grouped_gfp['GFP']],
    y = df_shape_factors_grouped_gfp['Shape Factor'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_shape_factors_grouped_nogfp_neighbours['Genotype'],df_shape_factors_grouped_nogfp_neighbours['GFP']],
    y = df_shape_factors_grouped_nogfp_neighbours['Shape Factor'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_shape_factors_grouped_nogfp['Genotype'],df_shape_factors_grouped_nogfp['GFP']],
    y = df_shape_factors_grouped_nogfp['Shape Factor'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_shape_factors_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_shapefactor'], row['GFP2_shapefactor']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_shapefactor'], row['Shape Factor']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Shape Factor',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    #xaxis=dict(tcka)
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Shape Factors.svg")



###
### aspect ratio
###


print(df_aspect_ratios_grouped_nocre)
fig = go.Figure(data=go.Scatter(
    x = [df_aspect_ratios_grouped_nocre['Genotype'],df_aspect_ratios_grouped_nocre['GFP']],
    y = df_aspect_ratios_grouped_nocre['Aspect Ratio'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_aspect_ratios_grouped_gfp['Genotype'],df_aspect_ratios_grouped_gfp['GFP']],
    y = df_aspect_ratios_grouped_gfp['Aspect Ratio'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_aspect_ratios_grouped_nogfp_neighbours['Genotype'],df_aspect_ratios_grouped_nogfp_neighbours['GFP']],
    y = df_aspect_ratios_grouped_nogfp_neighbours['Aspect Ratio'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_aspect_ratios_grouped_nogfp['Genotype'],df_aspect_ratios_grouped_nogfp['GFP']],
    y = df_aspect_ratios_grouped_nogfp['Aspect Ratio'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_aspect_ratios_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_aspectratio'], row['GFP2_aspectratio']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_aspectratio'], row['Aspect Ratio']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Aspect Ratio',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    #xaxis=dict(tcka)
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Aspect Ratios.svg")

###
### mean intensities
###


print(df_mean_intensities_grouped_nocre)
fig = go.Figure(data=go.Scatter(
    x = [df_mean_intensities_grouped_nocre['Genotype'],df_mean_intensities_grouped_nocre['GFP']],
    y = df_mean_intensities_grouped_nocre['Mean Intensity'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_mean_intensities_grouped_gfp['Genotype'],df_mean_intensities_grouped_gfp['GFP']],
    y = df_mean_intensities_grouped_gfp['Mean Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_mean_intensities_grouped_nogfp_neighbours['Genotype'],df_mean_intensities_grouped_nogfp_neighbours['GFP']],
    y = df_mean_intensities_grouped_nogfp_neighbours['Mean Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_mean_intensities_grouped_nogfp['Genotype'],df_mean_intensities_grouped_nogfp['GFP']],
    y = df_mean_intensities_grouped_nogfp['Mean Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_mean_intensities_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_mean_intensity'], row['GFP2_mean_intensity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_mean_intensity'], row['Mean Intensity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Mean GFP intensity',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    #xaxis=dict(tcka)
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Mean GFP intensity.svg")

###
### mean intensities
###


print(df_solidities_grouped_nocre)
fig = go.Figure(data=go.Scatter(
    x = [df_solidities_grouped_nocre['Genotype'],df_solidities_grouped_nocre['GFP']],
    y = df_solidities_grouped_nocre['Solidity'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))


fig.add_trace(go.Scatter(
    x = [df_solidities_grouped_gfp['Genotype'],df_solidities_grouped_gfp['GFP']],
    y = df_solidities_grouped_gfp['Solidity'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_solidities_grouped_nogfp_neighbours['Genotype'],df_solidities_grouped_nogfp_neighbours['GFP']],
    y = df_solidities_grouped_nogfp_neighbours['Solidity'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))


fig.add_trace(go.Scatter(
    x = [df_solidities_grouped_nogfp['Genotype'],df_solidities_grouped_nogfp['GFP']],
    y = df_solidities_grouped_nogfp['Solidity'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_solidities_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_solidity'], row['GFP2_solidity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_solidity'], row['Solidity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
    
fig.update_layout(
    yaxis_title='Solidity',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    #xaxis=dict(tcka)
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Solidity.svg")

###
### zo1 intensities
###


fig = go.Figure(data=go.Scatter(
    x = [df_zo1_intensities_grouped_nocre['Genotype'],df_zo1_intensities_grouped_nocre['GFP']],
    y = df_zo1_intensities_grouped_nocre['Zo1 Intensity'],
    mode='markers',
    marker=dict(
        size=8,
        color='Black',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_zo1_intensities_grouped_gfp['Genotype'],df_zo1_intensities_grouped_gfp['GFP']],
    y = df_zo1_intensities_grouped_gfp['Zo1 Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='darkgreen',
        symbol = 'square'
    )
))

fig.add_trace(go.Scatter(
    x = [df_zo1_intensities_grouped_nogfp_neighbours['Genotype'],df_zo1_intensities_grouped_nogfp_neighbours['GFP']],
    y = df_zo1_intensities_grouped_nogfp_neighbours['Zo1 Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='orange',
        symbol = 'square'
        )
))

fig.add_trace(go.Scatter(
    x = [df_zo1_intensities_grouped_nogfp['Genotype'],df_zo1_intensities_grouped_nogfp['GFP']],
    y = df_zo1_intensities_grouped_nogfp['Zo1 Intensity'],mode='markers',
    marker=dict(
        size=8,
        color='darkred',
        symbol = 'square'
        )
))

for idx,row in df_zo1_intensities_grouped_join.iterrows():
    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP2']]],
        y = [row['GFP3_zo1_intensity'], row['GFP2_zo1_intensity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    fig.add_trace(go.Scatter(
        x = [[row['Genotype'],row['Genotype']],[row['GFP3'], row['GFP']]],
        y = [row['GFP3_zo1_intensity'], row['Zo1 Intensity']],
        mode = 'lines',
        line=dict(color='darkgrey',width=1)
    ))

    
fig.update_layout(
    yaxis_title='Zo1 intensity',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
        #range=[0.9,1],
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    font=dict(
        family="Arial",
        size=12,
        color="Black"),
    #xaxis=dict(tcka)
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=False,
)

fig.write_image("Zo1 Intensity.svg")

In [ ]:
df_areas_vs_gfp_neighbours = pd.DataFrame(data = all_areas_vs_gfp_neighbours,index = [files,cre]).transpose()
df_areas_vs_gfp_neighbours = pd.melt(df_areas_vs_gfp_neighbours)
df_areas_vs_gfp_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_areas_vs_gfp_neighbours.explode('Value')
df['area'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)

df.to_excel('all_areas_vs_number_of_gfp_neighbours.xlsx')
df = df[df['neighbours'] >0]


print(df.head())
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['area','neighbours'])
#print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()
print(df_CreFF_mean)

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['area','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

import plotly.express as px
#fig = px.scatter(df_CreFF, x = 'neighbours', y='area')


fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['area'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['area'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['area'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['area'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['area']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['area']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )


fig.update_layout(
    xaxis_title='Area (\u03BCm\u00B2)',
    yaxis_title = 'Number of eGFP positive cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=1,         
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_area_gfp_neighbours.svg")

In [ ]:
###
### aspect ratios

df_aspect_ratios_vs_gfp_neighbours = pd.DataFrame(data = all_aspect_ratios_vs_gfp_neighbours,index = [files,cre]).transpose()
df_aspect_ratios_vs_gfp_neighbours = pd.melt(df_aspect_ratios_vs_gfp_neighbours)
df_aspect_ratios_vs_gfp_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_aspect_ratios_vs_gfp_neighbours.explode('Value')
df['aspect_ratio'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)

df.to_excel('all_aspect_ratios_vs_number_of_gfp_neighbours.xlsx')
df = df[df['neighbours'] >0]


print(df.head())
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['aspect_ratio','neighbours'])
#print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()
print(df_CreFF_mean)

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['aspect_ratio','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

import plotly.express as px
#fig = px.scatter(df_CreFF, x = 'neighbours', y='aspect_ratio')


fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['aspect_ratio'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['aspect_ratio'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['aspect_ratio'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['aspect_ratio'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['aspect_ratio']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,2,0.05)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['aspect_ratio']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,2,0.05)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )


fig.update_layout(
    xaxis_title='Aspect Ratio',
    yaxis_title = 'Number of eGFP positive cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=1,         
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_aspect_ratio_gfp_neighbours.svg")

In [ ]:
###
### solidities

df_solidities_vs_gfp_neighbours = pd.DataFrame(data = all_solidities_vs_gfp_neighbours,index = [files,cre]).transpose()
df_solidities_vs_gfp_neighbours = pd.melt(df_solidities_vs_gfp_neighbours)
df_solidities_vs_gfp_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_solidities_vs_gfp_neighbours.explode('Value')
df['solidity'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)

df.to_excel('all_solidities_vs_number_of_gfp_neighbours.xlsx')
df = df[df['neighbours'] >0]


print(df.head())
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['solidity','neighbours'])
#print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()
print(df_CreFF_mean)

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['solidity','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

import plotly.express as px
#fig = px.scatter(df_CreFF, x = 'neighbours', y='solidity')


fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['solidity'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['solidity'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['solidity'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['solidity'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['solidity']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['solidity']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )


fig.update_layout(
    xaxis_title='Aspect Ratio',
    yaxis_title = 'Number of eGFP positive cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=1,         
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_solidity_gfp_neighbours.svg")

In [ ]:
###
### shape factors

df_shape_factors_vs_gfp_neighbours = pd.DataFrame(data = all_shape_factors_vs_gfp_neighbours,index = [files,cre]).transpose()
df_shape_factors_vs_gfp_neighbours = pd.melt(df_shape_factors_vs_gfp_neighbours)
df_shape_factors_vs_gfp_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_shape_factors_vs_gfp_neighbours.explode('Value')
df['shape_factor'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)

df.to_excel('all_shape_factors_vs_number_of_gfp_neighbours.xlsx')
df = df[df['neighbours'] >0]


print(df.head())
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['shape_factor','neighbours'])
#print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()
print(df_CreFF_mean)

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['shape_factor','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

import plotly.express as px
#fig = px.scatter(df_CreFF, x = 'neighbours', y='shape_factor')


fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['shape_factor'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['shape_factor'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['shape_factor'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['shape_factor'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['shape_factor']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['shape_factor']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )


fig.update_layout(
    xaxis_title='Aspect Ratio',
    yaxis_title = 'Number of eGFP positive cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=1,         
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_shape_factor_gfp_neighbours.svg")

In [ ]:
###
### zo1 intensities

df_zo1_intensities_vs_gfp_neighbours = pd.DataFrame(data = all_zo1_intensities_vs_gfp_neighbours,index = [files,cre]).transpose()
df_zo1_intensities_vs_gfp_neighbours = pd.melt(df_zo1_intensities_vs_gfp_neighbours)
df_zo1_intensities_vs_gfp_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_zo1_intensities_vs_gfp_neighbours.explode('Value')
df['zo1_intensity'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)

df.to_excel('all_zo1_intensities_vs_number_of_gfp_neighbours.xlsx')
df = df[df['neighbours'] >0]


print(df.head())
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['zo1_intensity','neighbours'])
#print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()
print(df_CreFF_mean)

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['zo1_intensity','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

import plotly.express as px
#fig = px.scatter(df_CreFF, x = 'neighbours', y='zo1_intensity')


fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['zo1_intensity'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['zo1_intensity'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['zo1_intensity'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['zo1_intensity'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['zo1_intensity']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['zo1_intensity']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )


fig.update_layout(
    xaxis_title='Aspect Ratio',
    yaxis_title = 'Number of eGFP positive cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=1,         
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_zo1_intensity_gfp_neighbours.svg")

In [ ]:

df_areas_vs_all_neighbours = pd.DataFrame(data = all_areas_vs_all_neighbours,index = [files,cre]).transpose()
df_areas_vs_all_neighbours = pd.melt(df_areas_vs_all_neighbours)
df_areas_vs_all_neighbours.columns = ['file', 'Genotype', 'Value']
df = df_areas_vs_all_neighbours.explode('Value')
df['area'], df['neighbours'] = zip(*df.pop('Value'))
#df = df.set_index(['Genotype'])
df = df.drop(['file'], axis = 1)
#means = df.groupby(['Genotype']).mean()
#print(means)
df = df[df['neighbours'] >1]
df_CreFF = df[df.Genotype == 'Cre;Fl/Fl']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['area','neighbours'])
print(df_CreFF)
df_CreFF_mean = df_CreFF.groupby('neighbours').mean()
df_CreFF_mean = df_CreFF_mean.reset_index()
df_CreFF_std = df_CreFF.groupby('neighbours').sem()

df_CreFN = df[df.Genotype == 'Cre;Fl/-']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['area','neighbours'])
df_CreFN_mean = df_CreFN.groupby('neighbours').mean()
df_CreFN_mean = df_CreFN_mean.reset_index()
df_CreFN_std = df_CreFN.groupby('neighbours').sem()

df_NoCre = df[df.Genotype == 'NoCre']
#df = df.reset_index().pivot(index=None, columns = 'Genotype', values = ['area','neighbours'])
df_NoCre_mean = df_NoCre.groupby('neighbours').mean()
df_NoCre_mean = df_NoCre_mean.reset_index()
df_NoCre_std = df_NoCre.groupby('neighbours').sem()

fig = go.Figure(data=go.Scatter(
        y=df_CreFF['neighbours'],
        x=df_CreFF['area'],
        name = 'Cre;Fl/Fl',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFF_std['area'],
#             visible=True),
    marker_color='blue',
    mode='markers'
    ))
fig.add_trace(go.Scatter(
        y=df_CreFN['neighbours'],
        x=df_CreFN['area'],
        name = 'Cre;Fl/-',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_CreFN_std['area'],
#             visible=True),
    marker_color='orange',
    mode='markers',
    ))

fig.add_trace(go.Scatter(
        y=df_NoCre['neighbours'],
        x=df_NoCre['area'],
        name = 'No Cre',
#         error_y=dict(
#             type='data',
#             symmetric=True,
#             array=df_NoCre_std['area'],
#             visible=True),
    marker_color='darkgrey',
    mode='markers',
    ))

from scipy import stats

# Generated linear fit

from numpy import arange,array,ones

xi = df_CreFF['area']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFF['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(line)
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkblue',
                          name='trend all')
                          )

xi = df_CreFN['area']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_CreFN['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='darkorange',
                          name='trend all')
                          )

xi = df_NoCre['area']
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,df_NoCre['neighbours'])
print("slope: %f    intercept: %f r: %f p: %f" % (slope, intercept, r_value, p_value))
xj = arange(0,150)

line = slope*xj+intercept
#print(slope)
fig.add_traces(go.Scatter(x=xj, y=line,
                          mode = 'lines',
                          marker_color='black',
                          name='trend all')
                          )



fig.update_layout(
    xaxis_title='Area (\u03BCm\u00B2)',
    yaxis_title = 'Number of cell neighbours',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(240, 240, 240)',
    showlegend=True,
)

fig.update_traces(marker=dict(size=2,
                              
                             )
                 )
#fig.update_xaxes(range=[1.5, 7.5])
fig.write_image("xy_CreFF_area_all_neighbours.svg")

In [ ]:
#print(combined_aspect_ratios)
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
fig = go.Figure()
fig.add_trace(go.Box(y=df_nogfp_aspect_ratios_pv['NoCre'],name="No Cre",fillcolor="white",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_aspect_ratios_pv['Cre;Fl/Fl'],name="Cre Fl/Fl \ EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_aspect_ratios_pv['Cre;Fl/Fl'],name="Cre Fl/Fl tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_aspect_ratios_pv['Cre;Fl/-'],name="Cre Fl/- EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_aspect_ratios_pv['Cre;Fl/-'],name="Cre Fl/- tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.update_traces(boxpoints=False)
fig.update_layout(
    yaxis_title='Aspect Ratio',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    showlegend=False,
)

#fig.show()



fig.write_image("box_aspect_ratios.png")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
fig = go.Figure()
fig.add_trace(go.Box(y=df_nogfp_areas_pv['NoCre'],name="No Cre",fillcolor="white",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_areas_pv['Cre;Fl/Fl'],name="Cre Fl/Fl \ EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_areas_pv['Cre;Fl/Fl'],name="Cre Fl/Fl tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_areas_pv['Cre;Fl/-'],name="Cre Fl/- EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_areas_pv['Cre;Fl/-'],name="Cre Fl/- tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.update_traces(boxpoints='outliers')
fig.update_layout(
    yaxis_title='Area (\u03BCm\u00B2)',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    showlegend=False,
)

#fig.show()
print(np.nanmax((df_gfp_areas_pv['Cre;Fl/-'].values)))
fl_stats = stats.ttest_ind(df_gfp_areas_pv['Cre;Fl/Fl'].dropna().values, df_nogfp_areas_pv['Cre;Fl/Fl'].dropna().values, equal_var = True)
print(fl_stats)

flnull_stats = stats.ttest_ind(df_gfp_areas_pv['Cre;Fl/-'].dropna().values, df_nogfp_areas_pv['Cre;Fl/-'].dropna().values, equal_var = True)
print(flnull_stats)


fig.write_image("box_areas.png")

In [ ]:
##shape factors
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
fig = go.Figure()
fig.add_trace(go.Box(y=df_nogfp_shape_factors_pv['NoCre'],name="No Cre",fillcolor="white",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_shape_factors_pv['Cre;Fl/Fl'],name="Cre Fl/Fl \ EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_shape_factors_pv['Cre;Fl/Fl'],name="Cre Fl/Fl tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_shape_factors_pv['Cre;Fl/-'],name="Cre Fl/- EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_shape_factors_pv['Cre;Fl/-'],name="Cre Fl/- tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.update_traces(boxpoints=False)
fig.update_layout(
    yaxis_title='Shape Factor',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    showlegend=False,
)

#fig.show()
print(np.nanmax((df_gfp_shape_factors_pv['Cre;Fl/-'].values)))
fl_stats = stats.ttest_ind(df_gfp_shape_factors_pv['Cre;Fl/Fl'].dropna().values, df_nogfp_shape_factors_pv['Cre;Fl/Fl'].dropna().values, equal_var = True)
print(fl_stats)

flnull_stats = stats.ttest_ind(df_gfp_shape_factors_pv['Cre;Fl/-'].dropna().values, df_nogfp_shape_factors_pv['Cre;Fl/-'].dropna().values, equal_var = True)
print(flnull_stats)


fig.write_image("box_shape_factors.png")

In [ ]:
##solidity
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
fig = go.Figure()
fig.add_trace(go.Box(y=df_nogfp_solidities_pv['NoCre'],name="No Cre",fillcolor="white",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_solidities_pv['Cre;Fl/Fl'],name="Cre Fl/Fl \ EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_solidities_pv['Cre;Fl/Fl'],name="Cre Fl/Fl tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_solidities_pv['Cre;Fl/-'],name="Cre Fl/- EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_solidities_pv['Cre;Fl/-'],name="Cre Fl/- tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.update_traces(boxpoints=False)
fig.update_layout(
    yaxis_title='Shape Factor',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    showlegend=False,
)

#fig.show()
print(np.nanmax((df_gfp_solidities_pv['Cre;Fl/-'].values)))
fl_stats = stats.ttest_ind(df_gfp_solidities_pv['Cre;Fl/Fl'].dropna().values, df_nogfp_solidities_pv['Cre;Fl/Fl'].dropna().values, equal_var = True)
print(fl_stats)

flnull_stats = stats.ttest_ind(df_gfp_solidities_pv['Cre;Fl/-'].dropna().values, df_nogfp_solidities_pv['Cre;Fl/-'].dropna().values, equal_var = True)
print(flnull_stats)


fig.write_image("box_solidities.png")

In [ ]:
##zo1 intensity
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
fig = go.Figure()
fig.add_trace(go.Box(y=df_nogfp_zo1_intensities_pv['NoCre'],name="No Cre",fillcolor="white",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_zo1_intensities_pv['Cre;Fl/Fl'],name="Cre Fl/Fl \ EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_zo1_intensities_pv['Cre;Fl/Fl'],name="Cre Fl/Fl tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_gfp_zo1_intensities_pv['Cre;Fl/-'],name="Cre Fl/- EGFP",fillcolor="green",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.add_trace(go.Box(y=df_nogfp_zo1_intensities_pv['Cre;Fl/-'],name="Cre Fl/- tdTom",fillcolor="red",line=dict(width=1,color='black'),whiskerwidth=0.2))
fig.update_traces(boxpoints=False)
fig.update_layout(
    yaxis_title='Zo1 intensity',
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinecolor='rgb(0, 0, 0)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=100,
        r=80,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    showlegend=False,
)

#fig.show()
print(np.nanmax((df_gfp_zo1_intensities_pv['Cre;Fl/-'].values)))
fl_stats = stats.ttest_ind(df_gfp_zo1_intensities_pv['Cre;Fl/Fl'].dropna().values, df_nogfp_zo1_intensities_pv['Cre;Fl/Fl'].dropna().values, equal_var = True)
print(fl_stats)

flnull_stats = stats.ttest_ind(df_gfp_zo1_intensities_pv['Cre;Fl/-'].dropna().values, df_nogfp_zo1_intensities_pv['Cre;Fl/-'].dropna().values, equal_var = True)
print(flnull_stats)


fig.write_image("box_zo1_intensities.png")

In [ ]:
## make pretty figure of neighbours
import imageio
x=0

for k, elem in imgd.items():
    #print(key)
    #print(imgd[k][:])
    name = imgd[k]
    #print(name)
    import re
    name = re.sub('\.tif$', '', k)

    maski_orig = imgd[k][3]
    maski = clear_border(imgd[k][3], buffer_size=30,bgval=0, in_place=False)
    gfp_img_unscaled = imgd[k][1]
    zo1_img = imgd[k][2]

    #rescale gfp channel to 0 - 1 float
    gfp_img = rescale_intensity(gfp_img_unscaled.astype(float),in_range='image')

    #find unique cell IDs in mask
    uniqueValues = np.unique(maski)
    #print(len(uniqueValues))
    #initialise gfp pos/neg mask
    mask_posneg = np.zeros(maski.shape)
    mask_numneighbours = np.zeros(maski.shape)
    mask_num_gfp_neighbours = np.zeros(maski.shape)
    # initialise outlines array
    outlines_arr = np.zeros(maski.shape)

    outlines = plot.outlines_list(maski)
    # unique outlines values

    i = 1
    #print(outlines)
    for o in outlines:
        for p in o:
            #print(p)
            outlines_arr[p[1],p[0]] = uniqueValues[i]
        i += 1
    print(np.amax(outlines_arr))
    print(np.amax(maski))
    # intialise cell outlines array
    cell_outline = np.zeros(maski.shape)

    # initialise adjacency matrix
    adj = np.zeros((len(uniqueValues), len(uniqueValues)))

    # initialise EGFP list
    gfp = np.zeros(len(np.unique(maski))).astype(int)

    #rather than just using the complement of gfp to define gfp cells, we can define cells that we are confident that the cells are tdTom i.e. very little or no signal in gfp channel at all.
    # loop through uniquevalues of mask
    for j in range(len(uniqueValues)):
        # isolate individual cell mask
        cell_mask = maski == uniqueValues[j]
        # mask gfp channel by cell mask
        cell = np.where(cell_mask == True, gfp_img, np.nan)
        # find cell mask size
        cell_size = np.count_nonzero(~np.isnan(cell))

        # find number of pixels greater than an empirical threshold
        gfp_size = np.count_nonzero(cell > 0.08)
        nogfp_size = np.count_nonzero(cell < 0.06)

        # class cell as EGFP (3), confidently negative (1), or likely negative (2) depending on whether pixels are EGFP or negative
        if gfp_size >= cell_size*0.75:
            mask_posneg[maski == uniqueValues[j]] = 3
            gfp[j] = 3 # value denotes EGFP cell
            #print('cell is EGFP')

        elif nogfp_size >= cell_size*0.75: 
            mask_posneg[maski == uniqueValues[j]] = 1 # confident negative cell
            gfp[j] = 1 # value denotes a non-positive cell
            #print('cell is confidently tdTom')

        else:
            mask_posneg[maski == uniqueValues[j]] = 2 # likely negative cell
            gfp[j] = 2 # value denotes a likely non-positive cell
            #print('cell is likely tdTom')
    
       
        
        # define cell outline
        cell_label = np.where( maski == uniqueValues[j], 1, 0)
        # dilate cell outline
        #cell_dil = cell_label
        cell_dil = ndimage.binary_dilation(cell_label,iterations=3)

        # find neighbouring outlines 
        #print(np.amax(maski))
        masked_outlines = np.where(cell_dil == True,maski,0)
        neighbours = np.unique(masked_outlines)

        for k in neighbours:
            #print(k)
            if k != 0:
                m = np.argwhere(uniqueValues == k)
                #print(m[0][0])
                if m[0][0] != None:
                    #print(j, m[0][0])
                    adj[j, m[0][0]] = 1

    adj_sym = np.maximum( adj, adj.transpose() ).astype(int)    
    skimage.io.imsave(str(name) + '_mask_EGFP_positive_negative_255.jpg', mask_posneg.astype(np.int))
    skimage.io.imsave(str(name) + '_mask_EGFP_positive_negative.jpg', mask_posneg.astype(np.uint8))
    # indices of EGFP cells

    gfp_ind_indiv = np.where(gfp == 3)
    
    #print(gfp_ind_indiv)    
    
    num_neighbours = np.zeros(len(np.unique(maski))).astype(int)
    num_gfp_neighbours = np.zeros(len(np.unique(maski))).astype(int)
    
    for j in range(len(uniqueValues)):
        if gfp[j] == 1: #this means the cell in question is confidently tdTom
        
            num_gfp_neighbours[j] = np.count_nonzero(adj_sym[gfp_ind_indiv,j])
        
            #print(indiv_num_gfp_neighbours)
            num_neighbours[j] = np.count_nonzero(adj_sym[:,j])
            mask_num_gfp_neighbours[maski == uniqueValues[j]] = num_gfp_neighbours[j] + 1
        
        #print(num_neighbours[j])
        #print(gfp_num_neighbours[j])
        #else:
            #print('cell number ' + str(j) + ' is etdTom and is not a neighbour of EGFP cell ')
        #    gfp[j] = 0 # 0 now denotes tdTom cell with no EGFP neighbours
    mask_num_gfp_neighbours[maski == uniqueValues[0]] = 0
    skimage.io.imsave(str(name) + '_mask_number_of_egfp_neighbours_255.jpg', mask_num_gfp_neighbours.astype(np.int))
    skimage.io.imsave(str(name) + '_mask_number_of_egfp_neighbours.jpg', mask_num_gfp_neighbours.astype(np.uint8))
    
    num_gfp_neighbours[0] = 0
    num_neighbours[0] = 0
    print(num_gfp_neighbours)
    #max_idx = np.amax(num_gfp_neighbours)
    #print(num_gfp_neighbours[max_idx])
    #print(np.mean(num_gfp_neighbours))
    #print(gfp_num_neighbours)
    #print(np.mean(gfp_num_neighbours))

    all_ind.append([np.argwhere(gfp >=0).flatten().astype(int)])
    gfp_ind.append([np.argwhere(gfp == 3).flatten().astype(int)])
    nogfp_ind.append([np.argwhere(gfp == 1).flatten().astype(int)])
    
    neighbour.append ([num_neighbours])
    gfp_neighbour.append([num_gfp_neighbours])
    
    x+=1

    
    #print(nogfp_ind)
    
    #print(gfp_neighbours)
    #print(np.mean)

    #plt.imshow(gfp_img, cmap='inferno')
    #plt.imshow(outlines_arr>0, cmap='Reds', alpha = 0.5)
    #plt.imshow(mask_posneg==3, cmap='gray', alpha=0.2)

In [ ]:
plt.imshow(gfp_img, cmap='inferno')
#plt.imshow(outlines_arr>0, cmap='Reds', alpha = 0.5)
plt.imshow(maski == 
           885, alpha=0.5)